# Tools
## Introduction
- In this lesson, we'll hook up tools to generation.

## Installation

In [8]:
%pip install -q openai anthropic ipywidgets colorama
import os
os.environ['XDG_RUNTIME_DIR']="/tmp"
os.environ['INSPECT_EVAL_MODEL'] = "openai/gpt-4o-mini"

from helpers.reporter.pretty import pretty_results


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Adding tools
- We can add tools as part of the solvers.
- Inspect AI has a lot of tools build in. <https://inspect.aisi.org.uk/tools.html>
- Here we add the `bash` and `text_editor` as a tool
- The generate function can now use these tools.
- By default , the generate function goes into a loop to solve the question in the Sample.
- Here we limit it to a single loop for demonstration purposes.

Sandbox:
- Note that we run this Task in a `sandbox`.
- We set docker as our sandbox system.
- By default it pulls a default container.
- We've overriden the default container in the `compose.yaml` where we modified the container. More on this later


In [9]:
from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate, use_tools
from inspect_ai.scorer import model_graded_fact

from inspect_ai.tool import bash, text_editor

@task
def basic_tools() -> Task:

    dataset=[
        Sample(
            input="Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.",
            target="The generated code should have the filename hello-world.js",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            use_tools(bash(), text_editor()), 
            generate(tool_calls="single")
        ],
        scorer=[
            model_graded_fact()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(basic_tools, log_level="info",display="none")
print(pretty_results(results))

Output()

Status: success Model: openai/gpt-4o-mini
======= Sample Begin =================================================================
input : Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
target: The generated code should have the filename hello-world.js
 user       > Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
 assistant [tool:text_editor] > {'command': 'create', 'path': '/repo/hello-world.js', 'file_text': "console.log('Hello, World!');"}
 assistant  > 
 tool[text_editor] > 
======== Score =======================================================================
Scorer[model_graded_fact][VALUE]: I
Scorer[model_graded_fact][EXPLANATION]: To evaluate the submission against the expert's answer, we follow these steps:

1. **Identify the Content of the Expert Answer**: The expert answer states that "The generated code should have the filename hello-world.js." This clearly indicates t

## ReAct loop
- A more flexible way or running a tools loop is using a react solver. 

In [10]:
from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate
from inspect_ai.scorer import model_graded_fact

from inspect_ai.agent import react
from inspect_ai.tool import bash, text_editor, web_browser

@task
def react_basic_tools() -> Task:

    dataset=[
        Sample(
            input="Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.",
            target="The generated code should have the filename hello-world.js",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            react(
                tools=[bash(), text_editor()]
            ),
        ],
        scorer=[
            model_graded_fact()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(react_basic_tools, log_level="info",display="none")
print(pretty_results(results))

Output()

Status: success Model: openai/gpt-4o-mini
======= Sample Begin =================================================================
input : Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
target: The generated code should have the filename hello-world.js
 system     > 
You are a helpful assistant attempting to submit the best possible answer.
You have several tools available to help with finding the answer. You will
see the result of tool calls right after sending the message. If you need
to perform multiple actions, you can always send more messages with additional
tool calls. Do some reasoning before your actions, describing what tool calls
you are going to use and how they fit into your plan.

When you have completed the task and have an answer, call the submit()
tool to report it.

 user       > Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
 assistant [tool:text_editor] > {'co

## Web browser
- We can also add a web browser.
- This is not just a tool , but a tool source. Therefore we need to add it a little bit different.

In [11]:
from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate
from inspect_ai.scorer import model_graded_fact

from inspect_ai.agent import react
from inspect_ai.tool import bash, text_editor, web_browser

@task
def react_basic_and_web_tools() -> Task:

    dataset=[
        Sample(
            input="Generate a javascript file name hello-world.js. Make sure to check the file was successfully created. Search the web for the best practices to create a javascript file.",
            target="The generated code should have the filename hello-world.js. ",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            react(
                tools=[bash(), text_editor()]+ web_browser(interactive=False), # special case for web_browser
            ),
        ],
        scorer=[
            model_graded_fact()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(react_basic_and_web_tools, log_level="info",display="none")
print(pretty_results(results))

Output()

Status: success Model: openai/gpt-4o-mini
======= Sample Begin =================================================================
input : Generate a javascript file name hello-world.js. Make sure to check the file was successfully created. Search the web for the best practices to create a javascript file.
target: The generated code should have the filename hello-world.js. 
 system     > 
You are a helpful assistant attempting to submit the best possible answer.
You have several tools available to help with finding the answer. You will
see the result of tool calls right after sending the message. If you need
to perform multiple actions, you can always send more messages with additional
tool calls. Do some reasoning before your actions, describing what tool calls
you are going to use and how they fit into your plan.

When you have completed the task and have an answer, call the submit()
tool to report it.

 user       > Generate a javascript file name hello-world.js. Make sure to check th

## Custom tool
- It uses the description and types to send the schema over to the llm.


In [12]:
from inspect_ai.tool import tool

@tool
def add_tool():
    async def execute(x: int, y: int):
        """
        Add two numbers.

        Args:
            x: First number to add.
            y: Second number to add.

        Returns:
            The sum of the two numbers.
        """
        return x + y

    return execute


from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate
from inspect_ai.scorer import match

from inspect_ai.agent import react
from inspect_ai.tool import bash, text_editor, web_browser

@task
def custom_tool_addition() -> Task:

    dataset=[
        Sample(
            input="Add 10 + 20 and 30 + 40. Make sure to sum all of the sums.",
            target="2",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            use_tools(add_tool()),
            generate(),
        ],
        scorer=[
            match()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(custom_tool_addition, log_level="info",display="none")
print(pretty_results(results))

Output()

Status: success Model: openai/gpt-4o-mini
======= Sample Begin =================================================================
input : Add 10 + 20 and 30 + 40. Make sure to sum all of the sums.
target: 2
 user       > Add 10 + 20 and 30 + 40. Make sure to sum all of the sums.
 assistant [tool:add_tool] > {'x': 10, 'y': 20}
 assistant [tool:add_tool] > {'x': 30, 'y': 40}
 assistant  > 
 tool[add_tool] > 30
 tool[add_tool] > 70
 assistant [tool:add_tool] > {'x': 30, 'y': 70}
 assistant  > 
 tool[add_tool] > 100
 assistant  > The sum of 10 + 20 is 30, and the sum of 30 + 40 is 70. Adding those results together gives us a final sum of 100.
======== Score =======================================================================
Scorer[match][VALUE]: I
Scorer[match][EXPLANATION]: The sum of 10 + 20 is 30, and the sum of 30 + 40 is 70. Adding those results together gives us a final sum of 100.
**** End Sample ******************************************************************
